In [22]:
import utils as utils
import importlib
import data
import pickle

import torch as th
import torch.nn as nn
import torch.nn.functional as F

importlib.reload(utils)
importlib.reload(data)

train_dataset_path = './data/tempo/train'
test_dataset_path = './data/tempo/test'

device = 'cuda' if th.cuda.is_available() else 'cpu'

In [12]:
# Model descrbied in the paper plus droput
class OnsetDetectionCNN(nn.Module):
    def __init__(self):
        super(OnsetDetectionCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=(3, 7))
        self.pool1 = nn.MaxPool2d(kernel_size=(3, 1))
        self.conv2 = nn.Conv2d(10, 20, kernel_size=(3, 3))
        self.pool2 = nn.MaxPool2d(kernel_size=(3, 1))
        self.fc1 = nn.Linear(20 * 7 * 8, 256)
        self.fc2 = nn.Linear(256, 1)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 20 * 7 * 8)
        x = self.dropout(F.relu(self.fc1(x)))  # Apply dropout
        x = self.sigmoid(self.fc2(x))
        return x

# Initialize the model
model = OnsetDetectionCNN()

In [15]:
device = 'cuda' if th.cuda.is_available() else 'cpu'
print(device)

cpu


In [16]:
model.load_state_dict(th.load('best_model.pth'))
with open('mean_std.pkl', 'rb') as file:
    data = pickle.load(file)
mean = data['mean']
std = data['std']

In [17]:
# Load the paths and then split them into train and test set (validation set in our case for now).
wav_files_paths_test, _, _, _ = utils.load_dataset_paths(test_dataset_path, is_train_dataset=False)
features_test, sample_rates_test = utils.preprocess_audio(wav_files_paths_test)

100%|██████████| 50/50 [00:01<00:00, 34.51it/s]


In [18]:
tempos = [utils.get_tempo(model, x, mean, std, device) for x in features_test]

In [20]:
pred = {}
for tempo, filename in zip(tempos, wav_files_paths_test):
    filename = filename.split('/')[-1].replace('.wav', '')
    pred[filename] = {'tempo': tempo}

In [21]:
pred

{'test48': {'tempo': [129.19921875, 103.359375]},
 'test49': {'tempo': [79.50721153846153, 120.18531976744185]},
 'test03': {'tempo': [99.38401442307692, 147.65625]},
 'test17': {'tempo': [86.1328125, 172.265625]},
 'test16': {'tempo': [86.1328125, 172.265625]},
 'test02': {'tempo': [101.33272058823529, 151.99908088235293]},
 'test14': {'tempo': [114.84375, 172.265625]},
 'test28': {'tempo': [139.6748310810811, 103.359375]},
 'test29': {'tempo': [68.90625, 103.359375]},
 'test01': {'tempo': [68.90625, 135.99917763157896]},
 'test15': {'tempo': [191.40625, 95.703125]},
 'test39': {'tempo': [117.45383522727273, 156.60511363636363]},
 'test11': {'tempo': [87.5926906779661, 178.20581896551724]},
 'test05': {'tempo': [79.50721153846153, 120.18531976744185]},
 'test04': {'tempo': [82.03125, 166.70866935483872]},
 'test10': {'tempo': [198.76802884615384, 86.1328125]},
 'test38': {'tempo': [64.599609375, 129.19921875]},
 'test06': {'tempo': [132.51201923076923, 89.10290948275862]},
 'test12': 